# Run a single experiment

In [1]:
from downsample_data import main
import os
import pandas as pd
import numpy  as np

In [2]:
experiments_directory = "small-experiments"
holdout_directory     = "small-holdouts"
csvs_directory        = "small-csvs"
holdout_percent       = 10

In [3]:
amounts = ["1", "3"]

In [4]:
main(amounts,
     seed=4,
     holdout_percent=holdout_percent,
     experiments_directory=experiments_directory, 
     holdout_directory=holdout_directory)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 483.44it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 474.52it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 842.57it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 929.59it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 140.67it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 16.31it/s]


### Train

In [5]:
for amount in amounts:
    print("Training amount: {}".format(amount,))
    os.system("""python retrain.py              \\
        --summaries_dir /tmp/retrain_logs       \\
        --image_dir small-experiments/{}        \\
        --log_level 1                           \\
        --csv_folder small-csvs                 \\
        --summaries_dir small-ckpts/{}          \\
        --output_graph small-ckpts/{}/graph.pb  \\
        --how_many_training_steps 300           \\
        --validation_percentage 0               \\
        --testing_percentage 0
        """.format(amount, amount, amount))

Training amount: 1
Training amount: 3


### Evaluate

In [6]:
for amount in amounts:
    print("Inferring amount: {}".format(amount,))
    os.system("""python label_folder.py      \\
        --folder small-holdouts              \\
        --prefix small-experiments_{}        \\
        --csv_folder small-csvs              \\
        --graph small-ckpts/{}/graph.pb

        """.format(amount, amount, amount))

Inferring amount: 1
Inferring amount: 3


### Analyse

In [7]:
def holdout_threshold (run, threshold = 0.5):
    df = pd.read_csv(f"{csvs_directory}/small-experiments_{run}-results.csv")
    df["correct"] = df.apply(lambda row: row["predicted_" + row["true_label"]] 
                             > threshold, 1)
    return df 

def holdout_accuracy_threshold (run, threshold = 0.5):
    df  = holdout_threshold(run, threshold)
    acc = df["correct"].sum() / len(df)
    return np.round(acc, 2)


def holdout_max (run):
    df = pd.read_csv(f"{csvs_directory}/small-experiments_{run}-results.csv")
    
    def was_right (row):
        cols = [c for c in row.keys() if c.startswith("predicted")]
        best = ""
        last = 0
        
        for c in cols:
            name = c.split("_")[1]
            v    = float(row[c])
            if v > last:
                best = name
                last = v
        
        return best == row["true_label"]
    
    df["correct"] = df.apply(was_right, 1)
    return df

def holdout_accuracy_max (run):
    df  = holdout_max(run)
    acc = df["correct"].sum() / len(df)
    return np.round(acc, 2)

In [8]:
for amount in amounts:
    print(amount)
    print("              max:", holdout_accuracy_max(amount))
    print("  threshold (0.5):", holdout_accuracy_threshold(amount))
    print("  threshold (0.8):", holdout_accuracy_threshold(amount, threshold=0.8))
    print("")

1
              max: 0.54
  threshold (0.5): 0.23
  threshold (0.8): 0.02

3
              max: 0.68
  threshold (0.5): 0.55
  threshold (0.8): 0.25

